In [2]:
import numpy as np
import tensorflow as tf
import os

from skimage.metrics import structural_similarity as ssim   
from PIL import Image
from tensorflow import keras
from keras import layers

2025-02-13 19:55:12.389471: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 19:55:12.618017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739476512.667058  426648 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739476512.700286  426648 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 19:55:12.980972: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
#Define the CBR (Convolutional, Batch Normalization and ReLU block)
class CBR(keras.Model):
    
    def __init__(self,filters):
        super().__init__()
        self.Convolution = layers.Conv2D(filters = filters, kernel_size = 3, strides = 1, padding = 'same', use_bias = False)
        self.BatchN = layers.BatchNormalization()
        self.relu = layers.ReLU()

    def call(self,inputs):
        x = self.Convolution(inputs)
        x = self.BatchN(x)
        return self.relu(x)    
    
#Define the Network
inputs = keras.Input(shape = (None,None,3))

CBR1 = CBR(64)(inputs)

CBR2 = CBR(64)(CBR1)

Downsampling1 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR2)
CBR3 = CBR(128)(Downsampling1)

CBR4 = CBR(128)(CBR3)

Downsampling2 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR4)
CBR5 = CBR(256)(Downsampling2)

CBR6 = CBR(256)(CBR5)

Downsampling3 = layers.MaxPooling2D(pool_size = (2,2), strides = 2)(CBR6)
CBR7 = CBR(512)(Downsampling3)

Upsampling1 = layers.Conv2DTranspose(filters = 256, kernel_size = 2, strides = 2, padding = 'same')(CBR7)
#Add the Skip connections
combined_images1 = layers.Concatenate(axis = -1)([Upsampling1,CBR6])
CBR8 = CBR(512)(combined_images1)

CBR9 = CBR(256)(CBR8)

Upsampling2 = layers.Conv2DTranspose(filters = 128, kernel_size = 2, strides = 2, padding = 'same')(CBR9)
combined_images2 = layers.Concatenate(axis = -1)([Upsampling2,CBR4])
CBR10 = CBR(256)(combined_images2)

CBR11 = CBR(128)(CBR10)

Upsampling3 = layers.Conv2DTranspose(filters = 64, kernel_size = 2, strides = 2, padding = 'same')(CBR11)
combined_images3 = layers.Concatenate(axis = -1)([Upsampling3,CBR2])
CBR12 = CBR(128)(combined_images3)

CBR13 = CBR(64)(CBR12)

output = layers.Conv2D(filters = 1, kernel_size = 3, strides = 1, padding = 'same')(CBR13)
Mask = keras.Model(inputs = inputs, outputs = output, name = 'Test')

2025-02-13 19:55:26.219056: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
#Load the training data
def load_dataset(top_dir = 'input_data'):
    images_dataset = []
    for name in sorted(os.listdir(top_dir)):
        img = np.array(Image.open(os.path.join(top_dir,name)))
        img = img[:,:,:3]
        images_dataset.append(img)
    return images_dataset

def load_dataset_g(top_dir = 'input_data'):
    images_dataset = []
    for name in sorted(os.listdir(top_dir)):
        img = np.array(Image.open(os.path.join(top_dir,name)))
        images_dataset.append(img)
    return images_dataset

In [5]:
#Load the dataset
rain_images_a = np.asarray(load_dataset(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/A_Qian_Cam_A+B/Real/rain_images'))
rain_images_b = np.asarray(load_dataset(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/B_Quan/Real+Synthetic/rain_images'))
rain_images = np.append(rain_images_a,rain_images_b,axis = 0)
del(rain_images_a)
del(rain_images_b)

# y_mask_a = np.asarray(load_dataset_g(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/A_Qian_Cam_A+B/Real/KwonMask'))
# y_mask_b = np.asarray(load_dataset_g(top_dir = '/data/lady6758/Parmeet_Datasets/NewDatasets/B_Quan/Real+Synthetic/KwonMask'))
# y_mask = np.append(y_mask_a,y_mask_b,axis = 0)
# del(y_mask_a)
# del(y_mask_b)

rain_images = np.float32(rain_images)
# y_mask = np.float32(y_mask)

rain_images /= 255
# y_mask /= 255

# np.random.seed(12345)
# np.random.shuffle(rain_images)

# np.random.seed(12345)
# np.random.shuffle(y_mask)

In [6]:
Mask.load_weights('/users/lady6758/Mayowa_4YP_Code/ReTraining_ReTesting/Raindrop_Mask/Fold1Retrained/attentionweights.weights.h5')

In [7]:
save_dir = '/users/lady6758/Mayowa_4YP_Code/ReTraining_ReTesting/GAN/ModelGeneratedKwonMask'
no_of_images = rain_images.shape[0]
for i in range(no_of_images):
    img1 = rain_images[i]
    img1 = np.expand_dims(img1,axis = 0)
    generated_mask = np.squeeze(np.squeeze(Mask.predict(img1),axis = -1),axis = 0)
    generated_mask_uint8 = (generated_mask * 255).astype("uint8")
    filename = f"{i:05d}_mask.png"
    filepath = os.path.join(save_dir, filename)
    Image.fromarray(generated_mask_uint8).save(filepath)

    

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 